In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### 한경 컨센서스 리포트 크롤링

In [2]:
sdate = '2022-03-03'
edate = '2022-03-04'
now_page = 1
url = f'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate={sdate}&edate={edate}&now_page={now_page}'

In [3]:
response = requests.get(url, headers={'User-Agent':'Gils'}) # headers를 가져와야 읽어옴
response

<Response [200]>

In [4]:
# decode는 한글 깨짐 방지
# decode를 쓰기 위해 text가 아닌 content로 불러옮
dom = BeautifulSoup(response.content.decode('euc-kr', 'replace'), 'html.parser') 

In [5]:
# 테이블의 데이터를 읽어옴
elements = dom.select('#contents > div.table_style01 > table > tbody > tr')
len(elements)

20

In [6]:
# 마지막 페이지 번호를 확인
end_page = dom.select_one('#contents > div.paging > a.btn.last').get('href')
end_page = int(end_page.split('now_page=')[-1])
end_page

6

In [7]:
# date, category, title, writeCompany, content 데이터 가져오기
element = elements[0]
data = {
    'date' : element.select('td')[0].text,
    'category' : element.select('td')[1].text,
    'title' : element.select_one('td.text_l > a').text,
    'writeCompany' : element.select('td')[4].text, 
    'content' : 'http://consensus.hankyung.com'+element.select_one('td.text_l > a').get('href'),
}
data

{'date': '2022-03-04',
 'category': '시장',
 'title': '한화 주간전략',
 'writeCompany': '한화투자증권',
 'content': 'http://consensus.hankyung.com/apps.analysis/analysis.downpdf?report_idx=606147'}

In [8]:
# 데이터 DataFrame으로 변환
datas = [{
        'date' : element.select('td')[0].text,
        'category' : element.select('td')[1].text,
        'title' : element.select_one('td.text_l > a').text,
        'writeCompany' : element.select('td')[4].text, 
        'content' : 'http://consensus.hankyung.com'+element.select_one('td.text_l > a').get('href'),} for element in elements]

pd.DataFrame(datas)

,date,category,title,writeCompany,content
0,2022-03-04,시장,한화 주간전략,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
1,2022-03-04,산업,항공-해외 Top tier 방산업체 대비 높은 매출성장세를 이어간 4...,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
2,2022-03-04,산업,은행-1 월 은행 금리 : Base 금리의 시작점이 높다,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
3,2022-03-04,기업,BGF(027410) 그룹사 가치가 상승하기 시작,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
4,2022-03-04,기업,기아(000270) CEO Investor Day 후기 : 전기차 목표 상향과 실적...,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
5,2022-03-04,산업,철강-주요 업황 코멘트: 중국 2 월 철강 PMI 등,하이투자증권,http://consensus.hankyung.com/apps.analysis/an...
6,2022-03-04,시장,이진우의 전략에세이 2..0 Winner는 ‘주식’보다 ‘원자재’일...,메리츠증권,http://consensus.hankyung.com/apps.analysis/an...
7,2022-03-04,산업,운송 - meritz Transport Daily,메리츠증권,http://consensus.hankyung.com/apps.analysis/an...
8,2022-03-04,기업,에스원(012750) 이제 건물도 스마트하게 관리,하이투자증권,http://consensus.hankyung.com/apps.analysis/an...
9,2022-03-04,산업,음식료 - F&B Morning Talk,메리츠증권,http://consensus.hankyung.com/apps.analysis/an...


In [9]:
# 함수로 변환
def hankyung(sdate, edate) :
    now_page=1
    url = f'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate={sdate}&edate={edate}&now_page={now_page}'
    response = requests.get(url, headers={'User-Agent':'Gils'})
    
    dom = BeautifulSoup(response.content.decode('euc-kr', 'replace'), 'html.parser')
    
    end_page = dom.select_one('#contents > div.paging > a.btn.last').get('href')
    end_page = int(end_page.split('now_page=')[-1])
    
    dfs = []
    
    for i in range(1, end_page+1) :
        url = f'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate={sdate}&edate={edate}&now_page={i}'
        response = requests.get(url, headers={'User-Agent':'Gils'})

        dom = BeautifulSoup(response.content.decode('euc-kr', 'replace'), 'html.parser')
        
        elements = dom.select('#contents > div.table_style01 > table > tbody > tr')

        datas = [{
            'date' : element.select('td')[0].text,
            'category' : element.select('td')[1].text,
            'title' : element.select_one('td.text_l > a').text,
            'writeCompany' : element.select('td')[4].text, 
            'content' : 'http://consensus.hankyung.com'+element.select_one('td.text_l > a').get('href'),} for element in elements]
        
        dfs.append(pd.DataFrame(datas))
        
    return pd.concat(dfs, ignore_index=True)

In [10]:
df = hankyung('2022-03-03', '2022-03-04')

In [11]:
df

,date,category,title,writeCompany,content
0,2022-03-04,시장,한화 주간전략,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
1,2022-03-04,산업,항공-해외 Top tier 방산업체 대비 높은 매출성장세를 이어간 4...,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
2,2022-03-04,산업,은행-1 월 은행 금리 : Base 금리의 시작점이 높다,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
3,2022-03-04,기업,BGF(027410) 그룹사 가치가 상승하기 시작,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
4,2022-03-04,기업,기아(000270) CEO Investor Day 후기 : 전기차 목표 상향과 실적...,한화투자증권,http://consensus.hankyung.com/apps.analysis/an...
...,...,...,...,...,...
109,2022-03-03,기업,"티씨케이(064760) 독점 프리미엄 훼손, 하지만 여전히 1등",IBK투자증권,http://consensus.hankyung.com/apps.analysis/an...
110,2022-03-03,산업,인터넷-IBKS Daily [인터넷/게임],IBK투자증권,http://consensus.hankyung.com/apps.analysis/an...
111,2022-03-03,기타,Start with IBKS,IBK투자증권,http://consensus.hankyung.com/apps.analysis/an...
112,2022-03-03,기업,스카이라이프(053210) 변화의 긍정적 효과로 주가 회복 예상,IBK투자증권,http://consensus.hankyung.com/apps.analysis/an...
